# Train & Align NN AM with Win-to-MFCC and simple speaker adaptation
Repeatedly re-align phone labels sequence while training the phones model.
To avoid proliferation of the more frequent phones (and mostly the silence), we diminish b() probabilities of frequent phones during re-alignment. We use 3 states pre phone.

This version of training adds simple speaker adaptation by making average cepstra over the recording visible as NN inputs (very simple i-vector like approach). We split MFCC to 4 groups according to log-energy (split to above average and below average, then each group is further split into two the same way).

(Future versions might compute MFCC averages for key frequent phones.)

# Config

In [1]:
infile = 'sv400-500_training_0016.tsv'
sideview = 17 # how many additional MFCC frames before and after the focus point are seen
mid_size = 500
filename_base_base = "sv500s17_training"

In [2]:
%run ../prongen/hmm_pron.py --in-jupyter
%run ../acmodel/plot.py
%matplotlib ipympl
%run ../acmodel/matrix.py
%run ../acmodel/praat_ifc.py
%run ../acmodel/hmm_acmodel.py

device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(f"Using {device} device")

%run ../acmodel/nn_acmodel.py

hmm_pron.py library - generate Czech pron HMM. Included to this notebook.


Using cuda device


## Get training data
We previously aligned Czech CommonVoice train set using an ultra-prinmitive HMM/GMM and then NNs. Let's continue on it.

In [3]:
#infile = "mega4_training_0021.tsv"
#infile = 'nn_train.tsv'
#infile = 'sv200con_training_0004.tsv'
#infile = 'sv200c-300_training_0027.tsv'

df = pd.read_csv(infile, sep="\t", keep_default_na=False)
hmms = get_training_hmms(infile, derivatives=0)
b_log_corr = b_log_corrections(infile) # get b() corrections based on frequency

In [4]:
# Upgrade to 3 states per phone (just for duration, b() is still shared)
for hmm in hmms:
    triple_hmm_states(hmm)

In [5]:
all_mfcc, all_targets, b_set = collect_training_material(hmms)

out_size = len(b_set)
in_size = hmms[0].mfcc.size(1)

" ".join(b_set), out_size, in_size

('? A E G H N O Z a b c d e f g h j k l m n o p r s t u v y z | á é ó ú ý č ď ň Ř ř š ť Ž ž',
 45,
 13)

## Add speaker vectors (mean cepstra in 4 energy bands)

In [6]:
# Make s-vectors
all_speaker_vectors_refs = []
for hmm in hmms:
    hmm.speaker_vector = mfcc_make_speaker_vector(hmm.mfcc)
    ref = hmm.speaker_vector.to(device)
    all_speaker_vectors_refs += [ref]*len(hmm.mfcc)

## Changes for the Window-to-MFCC input

In [7]:
in_size = hmms[0].mfcc.size(1) * (sideview+1+sideview) + 4*13 # added s-vector
in_size

507

In [8]:
# for alignment decoding, change mfcc in all hmms (for training, we already have a copy)
# NOTE: Make speaker vectors BEFORE this!
for hmm in hmms:
    hmm.mfcc = mfcc_win_view(mfcc_add_sideview(hmm.mfcc, sideview=sideview), sideview=sideview)

## Setup training

In [9]:
model = NeuralNetwork(in_size, out_size, mid_size).to(device) # 50 20 100=svec 50=sv50
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=507, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): ReLU()
    (6): Linear(in_features=500, out_features=45, bias=True)
  )
)


In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

training_data = SpeechDataset(all_mfcc, all_targets, b_set, sideview=sideview, speaker_vectors=all_speaker_vectors_refs) # initial alignment

In [ ]:
for mega_epoch in range(0, 30):
    print(f"======= Train {filename_base_base}, Epoch {mega_epoch} ========")

    all_targets = "".join([hmm.targets for hmm in hmms])  # collect alignments
    training_data.all_targets = all_targets  # just update the object with new targets

    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True) # new dataloader for this alignment

    train_n_epochs(train_dataloader, optimizer, model, criterion, 5)

    
    filename_base = f"{filename_base_base}_{'%04d' % mega_epoch}"
    torch.save(model.state_dict(), filename_base+".pth")

    #break

    print('Interrupted training for re-alignment...')

    model.eval() # switch to evaluation mode


    for idx, hmm in enumerate(hmms):
        if idx%1000==0:
            print(f"Align {idx}")   
        alp = align_hmm(hmm, model, b_set, b_log_corr=b_log_corr*1.0, group_tripled=True)


    df['targets'] = [hmm.targets for hmm in hmms]

    df.to_csv(filename_base+".tsv", sep="\t", index=False)
    
    b_log_corr = b_log_corrections(filename_base+".tsv") # get new b() corrections based on frequency

======= Train sv500s17_training, Epoch 0 ========
[1, 20000] loss: 1.062
[1, 40000] loss: 0.764
[1, 60000] loss: 0.680
[2, 20000] loss: 0.582
[2, 40000] loss: 0.565
[2, 60000] loss: 0.549
[3, 20000] loss: 0.499
[3, 40000] loss: 0.496
[3, 60000] loss: 0.491
[4, 20000] loss: 0.454
[4, 40000] loss: 0.457
[4, 60000] loss: 0.456
[5, 20000] loss: 0.422
[5, 40000] loss: 0.429
[5, 60000] loss: 0.431
Interrupted training for re-alignment...
Align 0
Align 1000
Align 2000
Align 3000
Align 4000
Align 5000
Align 6000
Align 7000
Align 8000
Align 9000
Align 10000
======= Train sv500s17_training, Epoch 1 ========
[1, 20000] loss: 0.354
[1, 40000] loss: 0.361
[1, 60000] loss: 0.363
[2, 20000] loss: 0.332
[2, 40000] loss: 0.342
[2, 60000] loss: 0.347
[3, 20000] loss: 0.318
[3, 40000] loss: 0.329
[3, 60000] loss: 0.335
[4, 20000] loss: 0.308
[4, 40000] loss: 0.320
[4, 60000] loss: 0.324
[5, 20000] loss: 0.299
[5, 40000] loss: 0.313
[5, 60000] loss: 0.319
Interrupted training for re-alignment...
Align 0
A